In [ ]:
import pandas as pd
import numpy as np 
#from faker import Faker
import re, json, logging , html, os 
from lib.config import connstr
from lib.context import context
from lib.utils import *  
from lib.script_generator import script_generator  
from lib.dbupdate_parser import dbupdate_parser   
ctx=context() 
ctx.logger.setLevel(logging.DEBUG) 
with open('config.json', 'r') as f: 
    ctx.config=json.loads(f.read())     
import warnings
warnings.filterwarnings('ignore') 
import shutil
import string, random
import requests
alphabet = string.ascii_letters + string.punctuation + string.whitespace
import datetime
from faker import Faker
def id_generator(size=6, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))
fake = Faker()

def cve():
    return 'CVE-2024-' + str(fake.unique.random_int(min=10000, max=99999))


In [ ]:
fake.paragraph(nb_sentences=2)
fake.slug()
fake.ipv6()

In [ ]:
file=f'{ctx.get_down_dir()}MFA.xlsx'  
df=pd.read_excel(file, sheet_name=int(1), header=0, usecols='A:Z' )   


In [ ]:
dff=pd.concat([df]*50, ignore_index=True)
dff['System ID (Optional)'] = dff['System ID (Optional)'].apply(lambda s: id_generator(10))
dff['High Value Asset ID (Optional)'] = dff['High Value Asset ID (Optional)'].apply(lambda s: id_generator(10))
dff['System Name'] = dff['System Name'].apply(lambda s: id_generator(10))
dff['System Description'] = dff['System Description'].apply(lambda s: fake.paragraph(nb_sentences=2))
dff.head()

In [ ]:
df.columns[2]

In [ ]:
dff.to_excel(f'{ctx.get_down_dir()}MFA_50.xlsx',  sheet_name='Sheet2', index=False) 

In [ ]:
sql="""
SELECT COLUMN_NAME
  FROM INFORMATION_SCHEMA.COLUMNS
 WHERE TABLE_NAME LIKE '%BOD2302%' AND (ORDINAL_POSITION > 8  )
ORDER BY ORDINAL_POSITION ASC
"""
df=sql_todf(sql, connstr)  
df